In [ ]:
%matplotlib inline
from IPython.display import display, Math, Markdown
from sympy import *
init_printing(use_latex='mathjax')
# TODO strip off todos during processing??
# TODO wonder if we can convert to latex by default??
# I guess the answer is here?? https://docs.sympy.org/latest/_modules/sympy/printing/str.html
# TOOD shit. text has to be displayed as markdown...
# shit. maybe wrap patterns in text, and everythin else is latex??
def mdisplay(fmt, **kwargs):
    display(Markdown(fmt.format(**{k: f'${latex(v)}$' for k, v in kwargs.items()})))
    
# mdisplay = lambda m: display(Math(m))

In [ ]:
import matplotlib.pyplot as plt
from sympy import diff as D, Function as F
from sympy.plotting import plot, plot_parametric
from sympy.solvers.ode import classify_sysode
from scipy.integrate import odeint

t = symbols('t', real=True)
p = F('p')(t)
q = F('q')(t)
v = F('v')(t)
half = Rational(1, 2)

# TODO how to break down this function in ipython notebook?
def solve_EL(L, inits=[], Tmax=None, steps=None, phase=False, d3=False):
    mdisplay("L = {L}", L=L)

    ptov = Eq(p, L.diff(v))
    mdisplay("p(v): {ptov}", ptov=ptov)
    vtops = solve(ptov, v)
    
    for vtop in vtops:
        mdisplay("v(p) = {vtop}", vtop=vtop)
        break

    H = (v * ptov.rhs - L).subs(v, vtop)
    mdisplay("H = {H}", H=H)

    system = [
        Eq(q.diff(t), H.diff(p)),
        Eq(p.diff(t), -H.diff(q)),
    ]
    mdisplay("EL: {s}", s=system)
    
    (dq, dp) = [system[0].rhs, system[1].rhs]
    (dq, dp) = (lambdify((q, p, t), dq), lambdify((q, p, t), dp))
    # display(dq, dp)
    
    def phase(U, t):
        [qqq, ppp] = U
        return [dq(qqq, ppp, t), dp(qqq, ppp, t)]
        
    ts = np.linspace(0, Tmax, steps)
    
    all_ts = []
    all_qs = []
    all_ps = []
    for q0, p0 in inits:
        sol = odeint(phase, [q0, p0], ts)
        all_ts.append(ts)
        all_qs.append(sol[:, 0])
        all_ps.append(sol[:, 1])
    # display(system)
    # ok, type3 makes sense


    if phase:
        if d3:
            from mpl_toolkits.mplot3d import Axes3D
            plt.gca(projection='3d')
        for ts, qs, ps in zip(all_ts, all_qs, all_ps):
            if d3:
                plt.plot(qs, ps, ts)
            else:
                plt.plot(qs, ps, color='green')
        plt.xlabel('q(t)')
        plt.ylabel('p(t)')
    else:
        for ts, qs, ps in zip(all_ts, all_qs, all_ps):
            plt.plot(ts, qs, color='blue')
            plt.plot(ts, ps, color='red')
        plt.xlabel('t')
        plt.ylabel('q,p')
    
    plt.grid()
    display(plt.show())
    return
    # display(q_sol, p_sol)
    # odeint((q_sol, p_sol), [])
    # sol = odeint(pend, y0, t, args=(b, c))
    # print(sol)
    
    
    s_q0, s_p0 = symbols('p_0 q_0', real=True)

    sol = dsolve(
        system, 
        [q, p],
        # ics={
        #     q.subs(t, 0): s_q0,
        #     p.subs(t, 0): s_p0,
        # }
    )
    [q_sol, p_sol] = sol
    display(q_sol, p_sol)
    
    # TODO ugh. ok, I guess try numerically
    (q_sol, p_sol) = (e.subs({s_q0: q0, s_p0: p0}) for e in (q_sol, p_sol))
    # display(q_sol, p_sol)
    #plot_parametric(
    #    (q_sol.rhs, p_sol.rhs),
    #    (t, 0, 10),
    #    xlabel='q',
    #    ylabel='p',
    #)
    # from sympy.utilities.lambdify import lambdify

    
L = half * v ** 2 - half * q ** 2

# L = half * v ** 2 + half * q ** 2
# hmm, interesting. it gets exponential very quickly!
# also, space is basically separated. if you happened to be in negative part, you can't get into positive?


# L = half * v ** 2 * q ** 2

# L  = exp(v) - q ** 2
# TODO interesting! look at slope field, this lagrangian is always doomed!!!
# TODO what if we continue it to complex plane?

# TODO shit. not invertible with repsect to v

# TODO TypeError: 'Equality' object is not iterable -- this is clearly not the expected error...
inits = [
    (-15, -15),
    (15, -15),
    (-15, 15),
    (15, 15),
]
q_i = 1
p_i = 1


# L  = exp(v) - q ** 2
inits = [
    #(-q_i , -p_i),
    (-20 , 100), # this guy lives a bit longer
    (-20 , 1),
    (0, 1),
    # (q_i  , p_i),
]
# TODO for infinite potential well: when (5, -5), we kick the ball right on top!
# (5, -5.02) -- it turns left
# (5, -4.98) -- it turns back right
# vertical band -- regions of space

# TODO v ** 3 is not uniquely invertible. is that a problem?
# TODO right. any more interesting functions??

# L = log(v) - q ** 2


# ok, here it just bumps into the boundary as well
# TODO wonder why is it so similar to exp??
# TODO also, need to convert back to v from p?
# TODO hm. logarithmic dies as well, but only defined for q > 0; p > 0

# L = 1 / v ** 2  - q ** 2
# this one is weird. if it doesn't get to p = 0 singularity, it's fine and just keeps goint to infty
inits = [
    #(-q_i , -p_i),
    # (-5 , 5), # this guy lives a bit longer
    (-10 , -100),
    (-5 , -100),
    (-1 , -100),
    # (q_i  , p_i),
]

# TODO wtf?? what's up with duplication?
solve_EL(
    L,
    inits=inits,
    Tmax=30,
    steps=1000,
    phase=True,
    # d3=True,
)
    

# TODO wolframs slope/direction plot is pretty cool. try to reproduce?
# TODO phase plot?
# TODO lambdify that and plot?


In [ ]:
# v, q, p = var('v q p')
# C = var('C')
# 
# L = 1/2 * v^2 + q * v - q^2
# 
# print("Lagrangian")
# L.show()
# H = (v * L.diff(v) - L)
# rep = (p == L.diff(v)).solve(v)[0]
# print("Hamiltonian")
# H.full_simplify().show()
#aaa = var('aa')
#display(Markdown(f'hi = {aaa}'))